# Explanation

This notebook is used to test the classes used and implemented in the transformer_classes.py file

Remember:
We need to implement the following (found in "Attention is all you need" paper available at https://arxiv.org/pdf/1706.03762.pdf):

- Transformer Architecture
- Scalled Dot Product
- Multi-Head Attention

And all other things required like positionnal encoding


# 1. Function Classes
## I. Build Vocab from Glove & Embedding

First we'll load and setup glove and our vocab to get the first "brick" for embedding

For this test I'll load all glove pretrained weight and build the vocab

#### 1st Version

Using Glove and the same data cleaning method as the one in the main branch which let lots of unknown words

In [2]:
# Main/global imports
import torch
import numpy as np
import pandas as pd

from textfn import *
from classes import *
from tranformer_classes import *

%load_ext autoreload
%autoreload 2

In [52]:
df = loadDts('dataset/train_processed.csv')

In [56]:
d_model = 50 # Renammed from embedding_dim
glove_path = 'glove_pretrained/glove.6B.{}d.txt'.format(d_model)
vocab_size = 10000
max_seq_length = 20

embeddings = np.zeros((vocab_size+2, d_model))
word_to_index = {}
index=0
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        if index >= vocab_size-2:
            break
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float16")
        embeddings[index] = vector
        word_to_index[word] = index
        index +=1 
    f.close()
embeddings[index+1] = np.zeros(d_model)
embeddings[index+2] = np.zeros(d_model)
word_to_index['<unk>'] = index+1
word_to_index['<pad>'] = index+2
vocab_size+=2

In [57]:
data = TranformerGloveDataset(df, max_seq_length, word_to_index, train=True)

### I. i. Embedding Handling

As we built the vocab, we'll need to handle the embedded values of the words.

#### Method 1:

Using Glove's pretrained weights and freezing the layer in the class Embedder

#### Method 2:

Idem as method 1 but putting this direclty in the Encoder class using the embedding layer

For now, using Method 1 

## II. Positional Encoding

Positional Encoding is a matrix which define the position of the word in the sentence.

It's defined with:

$$
PE_{(pos, 2i)} = sin(pos/10000^{2i/d_{model}})
$$
And
$$
PE_{(pos, 2i+1)} = cos(pos/10000^{2i/d_{model}})
$$

As stated on the original paper: 
"The positional encodings have the same dimension $d_{model}$ as the embeddings, so that the two can be summed."


So the dimensions of the PE matrix are the **sentence size** and **embedding size** or $d_{model}$

##### Method 1

Create a class that compute the positional encoding

In [8]:
# Example to check if positional encoder work
data = torch.randn(5, 10, 6)
pos_enc = PositionalEncoder(10, 6)
encoded_input = pos_enc(data)

print(encoded_input.size())

torch.Size([5, 10, 6])


## III. Multi-Head Attention & Scaled Dot-Product Attention 

Multi-Head Attention is the one of the "main" component of the transformer network.

It's using a set of matrices which will be trained to handle a specific role in the network:
- **Queries (Q)**: Relationship & Dependencies with tokens in sequence.
- **Keys (K)**: Key information used to compare against when computing scores.
- **Values (V)**: Weighted sum of the mechanism

Those matrices are made/initialized from inputs' embeddings vector with the positional encoding.


In the Multi-Head Attention, we split the embedding into multiple layers (or **heads**) where $N$ is the number of head. $d_k$ will be refering to the last dimension where $d_k = d_{model}/N$

**Dropout**: As the original paper state: "_We apply dropout to the sums of the embeddings and the positional encoding in both the encoder and decoder stacks. For the base model we use a rate of_ $P_{drop}=0.1$"

### III. i Scaled Dot-Product Attention

As the original paper stated, this Attention is computed as: $$Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

Without forgetting that we can add both mask and dropout to this.

#### Method 1 

Make the Multi-Head Attention class and the Attention (Scaled Dot-Product Attention) as a function in it.

#### Method 2 

Idem as method 1, but putting the Attention in a splitted class.




## IV. Feed-Forward Network

The Feed-Forward "layer" serve the purpose of deepens the whole networks by using Linear layers.

As stated in the original paper: "_This consists of two linear transformations with a ReLU activation in between_"

The number set per default of $d_{ff}$ is stated in original paper: "_[...] and the inner-layer has dimensionality_ $d_{ff} = 2048$"



## V. Normalization 

Normalization is important for our network, to prevent our values to not change too much, so model can train faster and better.

Original paper state that they're using Layer Normalization.
To implement LN, we need to implement the following:
$$LN(z;\alpha,\beta) = \frac{z-\mu}{\sigma}\odot \alpha + \beta$$

This can be found into the "Layer Normalization" paper, Page 13, 15 & 16th formula [HERE](https://arxiv.org/pdf/1607.06450.pdf)



# 2. Main blocks and Architecture

## I. Encoder/Decoder & Problem

While both Encoder & Decoder Blocks differ a bit:
- Encoder has "only" 1 Multi-Head Attention and 1 Feed Forward
- Decoder has 2 Multi-Head Attention and 1 Feed Forward and receive Encoder output

The common thing between the 2 is the skipped connections and the layers used.
So no particular difficulties in implementing the blocks.

With all previous classes implemented, we can make the parts classes by adding our embeddings/positional encoding and using either copy.deepcopy() or nn.ModuleList() to can create multiple independent blocks/module for our model to work with.

Althought the current transformer is "finished", the reference used implemented it for sequence to sequence, but Disaster Tweet is a Sentiment Analysis task, so we need to change a few things to make a sentiment analysis task from it.

## II. Changes for Sentiment Analysis

### i. Transformer class

As sentiment analysis is a Many-to-One setup, we don't need the Decoder part of the transformer.
So we we'll create a new class for sentiment analysis without decoder, adapt the output, Linear layer and forward computation for a binary output.

#### a. Different setups

There's different setups related to NLP:
- **Many-to-One**: Take a sequence and map it to two or more classes (Ex: Sentiment Analysis)
- **Many-to-Many**: Both inputs and output are sequences (Ex: Machine Translation)
- **One-to-Many**: Input is a single value and output a sequence (Ex: Image Captioning) 



## Annex

For this notebook and creation, I've used multiple sources:
- [How to code The Transformer in Pytorch - Toward Data Science - Samuel Lynn-Evans](https://towardsdatascience.com/how-to-code-the-transformer-in-pytorch-24db27c8f9ec) (As reference for sequence to sequence implementation)
- [Attention is All You Need](https://arxiv.org/abs/1706.03762) (As a base)
- [Layer Normalization](https://arxiv.org/pdf/1607.06450.pdf) (For layer norm)
- [ChatGPT](https://openai.com/chatgpt) (For comprehension/question and quick alternatives)
- Many Kaggle's Notebooks and Medium/Toward Data Science articles (To add to ChatGPT's response) 
